# Neural network approach on population dataset 

This Neural Network Approach for Truth Discovery was inspired by the model that was introuduced in 2017 by Jermaine Marshall, Arturo Argueta and Dong Wang in thier article titled "A Neural Network Approach for Truth Discovery in Social Sensing"

In [1]:
import os
import pandas as pd
import numpy as np
import os.path as op
from sklearn.model_selection import train_test_split
# KERAS LIBRARIES
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import time
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.insert(0, '../../models')
from PY.NN_model import get_model
import time

Using TensorFlow backend.


In [2]:
DATA_DIR = '../../../../Data'

In [3]:
# Load all the claims
claims_input = pd.read_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/claims_DL_1.csv'))
claims_input.head()

ObjectID      PropertyID  PropertyValue                  SourceID  \
0  abu dhabi  Population2006        1000230         0 (68.162.248.83)   
1  abu dhabi  Population2006        1850230  1513217: Mohammedfairouz   
2  amsterdam  Population2006         741329             141597: Ilse@   
3  amsterdam  Population2006         742884           1300620: Krator   
4   adelaide  Population2006        1124315         3922171: Pirate05   

   TimeStamp  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [4]:
# Load the ground truth
ground_truth = pd.read_csv(op.join(DATA_DIR, 'population/Claims_to_work_with/truths_DL_1.csv'))
ground_truth.head()

ObjectID      PropertyID  PropertyValue
0      cleveland, ohio  Population2000         478403
1        gary, indiana  Population2000         102746
2      flint, michigan  Population2000         124943
3  compton, california  Population2000          93493
4     washington, d.c.  Population2005         582049

In [5]:
# Build the ground_truth vector.
tc_indexes = dict()
tc_labels = np.array([], dtype=int)
sources = set()
for index, row in ground_truth.iterrows():
    tc_id = row["PropertyID"]+"|"+row["ObjectID"]+"|"+str(row["PropertyValue"])
    if tc_id not in tc_indexes:
        tc_indexes[tc_id] = tc_labels.size
        tc_labels = np.append(tc_labels, 1)
    filtered_claims = claims_input[claims_input["PropertyID"] == row["PropertyID"]]
    filtered_claims = filtered_claims[filtered_claims["ObjectID"] == row["ObjectID"]]
    
    for idx, claim in filtered_claims.iterrows():
        if str(claim["PropertyValue"]) != str(row["PropertyValue"]):
            tc_id = claim["PropertyID"]+"|"+claim["ObjectID"]+"|"+str(claim["PropertyValue"])
            if tc_id not in tc_indexes:
                tc_indexes[tc_id] = tc_labels.size
                tc_labels = np.append(tc_labels, 0)
        sources.add(claim["SourceID"])
            
    
print("Number of claims :", tc_labels.size)
print("Number of sources:", len(sources))

Number of claims : 697
Number of sources: 632


In [6]:
# Building the sensing matrix

claims_matrix = []

for source_num, source in enumerate(sources):  # foreach claims
    claim_vector = np.zeros(len(tc_indexes), dtype=int)       # initialize the vector with the claims
    
    source_claims = claims_input[claims_input["SourceID"] == source]
    for index, row in source_claims.iterrows():
        row_claim = row["PropertyID"]+"|"+row["ObjectID"]+"|"+str(row["PropertyValue"])
        if row_claim in tc_indexes:
            claim_vector[tc_indexes[row_claim]] = 1
    claims_matrix.append(claim_vector)
    if source_num % 200 == 0:
        print("Processed source %i " % source_num)
print("Done...")
# Our claim matrix is in the form specified above.

Processed source 0 
Processed source 200 
Processed source 400 
Processed source 600 
Done...


In [7]:
claims_matrix = np.array(claims_matrix, dtype=int)
print("Sensing Matrix shape: ", claims_matrix.shape)

Sensing Matrix shape:  (632, 697)


In [8]:
sensing_matrix_with_truth = claims_matrix.transpose()
tc_labels_shaped = tc_labels.reshape(sensing_matrix_with_truth.shape[0], 1)
claims_sources_with_truth = np.append(sensing_matrix_with_truth, tc_labels_shaped, axis=1)

In [9]:
# Defining constants...
random_seed1 = 7
# # INSTANCE'S CONSTANTS
sources_num = len(sources)
# # MODEL'S CONSTANTS
train_set_size = 0.9
test_set_size = 1-train_set_size
learning_rate = 0.025
batch_size = 20
epochs = 150  
fhl_units = sources_num
shl_units = int(sources_num/2)  # this is the default according to the paper
thl_units = shl_units

In [10]:
# # SPLIT IN TRAIN SET AND DATA SET.
train_set, test_set = train_test_split(claims_sources_with_truth, train_size=train_set_size, test_size=test_set_size, random_state=random_seed1)

In [11]:
# # EXTRACT LABELS (Yt) and SAMPLES (X) For TRAIN-SET
train_set_x = train_set[:, 0:train_set.shape[1]-1]
# print(train_set_x.shape)
train_set_y = train_set[:, train_set.shape[1]-1]
# print(train_set_y.shape)

In [12]:
# # EXTRACT LABELS (Yt) and SAMPLES (X) For TEST-SET
test_set_x = test_set[:, 0:test_set.shape[1]-1]
# print(test_set_x.shape)
test_set_y = test_set[:, test_set.shape[1]-1]
# print(test_set_y.shape)

In [13]:
X_final=np.concatenate((train_set_x,test_set_x))

In [14]:
y_final=np.concatenate((train_set_y,test_set_y))

In [15]:
start=time.time()
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores1 = []
cvscores2 = []
cvscores3 = []
cvscores4 = []
num_iter=0

for train, test in kfold.split(X_final, y_final):
    num_iter=num_iter+1
    model=get_model(fhl_units, shl_units, thl_units,sources_num,random_seed1,learning_rate)
    # Fit the model
    history=model.fit(X_final[train], y_final[train], epochs=epochs, batch_size=batch_size,verbose=0)
    # evaluate the model
    scores = model.evaluate(X_final[test], y_final[test], verbose=0)
    print("------subset number", num_iter,"------")
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores1.append(scores[1] * 100)
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    cvscores2.append(scores[2] * 100)
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    cvscores3.append(scores[3] * 100)
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
    cvscores4.append(scores[4] * 100)
print("--------------------------------------------" )   
print("accuracy","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores1), np.std(cvscores1)))
print("precision","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores2), np.std(cvscores2)))
print("recall","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores3), np.std(cvscores3)))
print("f1-mesure","%.2f%% (+/- %.2f%%)" % (np.mean(cvscores4), np.std(cvscores4)))
end=time.time()

------subset number 1 ------
accuracy: 81.43%
precision_mesure: 100.00%
recall_mesure: 57.50%
f1_mesure: 72.86%
------subset number 2 ------
accuracy: 87.14%
precision_mesure: 87.46%
recall_mesure: 92.86%
f1_mesure: 89.66%
------subset number 3 ------
accuracy: 82.86%
precision_mesure: 88.38%
recall_mesure: 81.44%
f1_mesure: 84.42%
------subset number 4 ------
accuracy: 88.57%
precision_mesure: 92.99%
recall_mesure: 74.29%
f1_mesure: 80.68%
------subset number 5 ------
accuracy: 71.43%
precision_mesure: 78.52%
recall_mesure: 49.79%
f1_mesure: 58.33%
------subset number 6 ------
accuracy: 78.57%
precision_mesure: 74.17%
recall_mesure: 79.26%
f1_mesure: 76.02%
------subset number 7 ------
accuracy: 85.71%
precision_mesure: 85.00%
recall_mesure: 85.56%
f1_mesure: 84.57%
------subset number 8 ------
accuracy: 84.06%
precision_mesure: 89.49%
recall_mesure: 74.99%
f1_mesure: 81.02%
------subset number 9 ------
accuracy: 76.81%
precision_mesure: 83.81%
recall_mesure: 62.70%
f1_mesure: 70.27%


In [16]:
print('time of execution', end-start)

time of execution 539.4955489635468
